# Clustering Crypto

In [2]:
!pip install hvplot


In [3]:
!pip install plotly

In [4]:
!pip install sklearn

  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2959 sha256=386ebeebd8f3d5c7e31989f10c93ece0890a4b5cb180b52623eccc2c274ee9c8
  Stored in directory: /Users/alexahoynacke/Library/Caches/pip/wheels/f8/e0/3d/9d0c2020c44a519b9f02ab4fa6d2a4a996c98d79ab2f569fa1
Successfully built sklearn


In [61]:
# Initial imports
import pandas as pd
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Deliverable 1: Preprocessing the Data for PCA

In [6]:
# Load the crypto_data.csv dataset.
file_path = "/Users/alexahoynacke/Desktop/CWRUBootcamp/Module 18 Unsup Machine Learning /Starter_Code/crypto_data.csv"
df = pd.read_csv(file_path, index_col = 0)
df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [8]:
# Keep all the cryptocurrencies that are being traded.
df1 = df.loc[df['IsTrading'] == True]
df1.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [9]:
# Keep all the cryptocurrencies that have a working algorithm.
df0 = df1.sort_values(by = 'Algorithm', ascending = False)
df0.tail()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
AQUA,Aquachain,Argon2,True,PoW,0.000000e+00,42000000
OPES,Opes,Argon2,True,PoW,NaN,52000000
BOAT,Doubloon,536,True,PoW/PoS,NaN,500000000
ESP,Espers,536,True,PoW/PoS,2.280188e+10,50000000000
HODL,HOdlcoin,1GB AES Pattern Search,True,PoW,1.144895e+07,81962100


In [13]:
# Remove the "IsTrading" column. 
df2 = df1.drop(['IsTrading'], axis = 1)
df2.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [15]:
# Remove rows that have at least 1 null value.
df3 = df2.dropna()
df3.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [17]:
# Keep the rows where coins are mined.
df4 = df3.loc[df3['TotalCoinsMined'] > 0]
df4.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [18]:
# Create a new DataFrame that holds only the cryptocurrencies names.
cc_names_df = df4[["CoinName"]]
cc_names_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [21]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = df4.drop(['CoinName'], axis = 1)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [22]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns = ['Algorithm', 'ProofType'])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# Standardize the data with StandardScaler().
from sklearn.preprocessing import StandardScaler
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:5])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Deliverable 2: Reducing Data Dimensions Using PCA

In [27]:
# Using PCA to reduce dimension to three principal components.
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components = 3)
pca


PCA(n_components=3)

In [28]:
# Create a DataFrame with the three principal components.
X_pca = pca.fit_transform(X_scaled)
X_pca

array([[-0.34167888,  0.95310667, -0.5423366 ],
       [-0.32502623,  0.9533589 , -0.54272856],
       [ 2.31152841,  1.69650528, -0.5643735 ],
       ...,
       [ 0.3386789 , -2.28844992,  0.37405597],
       [-0.16617605, -1.93025883,  0.38571276],
       [-0.30730844,  0.79268545, -0.21091069]])

In [32]:
pc_df = pd.DataFrame(data = X_pca, columns = ["PC1", "PC2", "PC3"], index = (X.index.tolist()))
pc_df.head()

,PC1,PC2,PC3
42,-0.341679,0.953107,-0.542337
404,-0.325026,0.953359,-0.542729
1337,2.311528,1.696505,-0.564374
BTC,-0.141191,-1.314976,0.179906
ETH,-0.140506,-2.034947,0.344013


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [40]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1,11))


Running K-Means with `k=4`

In [44]:
# Initialize the K-Means model.
from sklearn.cluster import KMeans
model = KMeans(
     n_clusters=4,
     max_iter=300,
     random_state=0
)

# Fit the model
model.fit(pc_df)

# Predict clusters
predictions = model.predict(pc_df)
print(predictions)
pc_df["Class"] = model.labels_

[0 0 0 3 3 3 0 3 3 3 0 3 0 0 3 0 3 3 0 0 3 3 3 3 3 0 3 3 3 0 3 0 3 3 0 0 3
 3 3 3 3 3 0 0 3 3 3 3 3 0 0 3 0 3 3 3 3 0 3 3 0 3 0 0 0 3 3 3 0 0 0 0 0 3
 3 3 0 0 3 0 3 0 0 3 3 3 3 0 0 3 0 3 3 0 0 3 0 0 3 3 0 0 3 0 0 3 0 3 0 3 0
 3 0 0 3 3 0 3 3 3 0 3 3 3 3 3 0 0 3 3 3 0 3 0 3 3 0 3 0 3 0 0 3 3 0 3 3 0
 0 3 0 3 0 0 0 3 3 3 3 0 0 0 0 0 3 3 0 0 0 0 0 3 0 0 0 0 0 3 0 3 0 0 3 0 3
 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 0 0 0 3 3 0 0 3 3 0 0 0 0 0 3 0 0 0 0 0 0
 0 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 3 0 3 0 0 3 0 3 3 0 3 3 0 3 0 0 0 3 0 0 3
 0 0 0 0 0 0 0 3 0 3 0 0 0 0 3 0 3 0 3 3 3 3 0 3 0 0 3 0 3 3 3 0 3 0 3 3 3
 0 3 0 3 0 0 1 3 0 3 3 3 3 3 0 0 3 0 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 3 0 0
 0 3 3 3 3 0 0 0 0 3 0 3 3 3 0 0 3 3 0 0 3 0 3 3 3 0 3 3 0 0 0 3 3 3 0 0 0
 3 3 0 3 3 3 3 0 1 0 3 3 3 0 1 0 0 0 0 3 3 3 3 0 0 0 3 0 3 0 0 0 0 3 0 0 3
 0 0 3 3 0 3 0 3 3 3 3 0 0 3 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 0 0 3 0 3 3 3 3
 0 0 0 0 3 0 0 3 0 0 3 1 3 0 3 3 0 0 3 0 3 3 0 3 3 0 3 0 3 0 0 3 0 0 0 0 0
 3 3 3 0 0 0 3 0 3 0 3 0 

In [45]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = crypto_df.join(pc_df, how = 'inner')
clustered_df.head()

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = clustered_df.join(cc_names_df, how = 'inner')
clustered_df.head()

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,Class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.341679,0.953107,-0.542337,0,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.325026,0.953359,-0.542729,0,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.311528,1.696505,-0.564374,0,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.141191,-1.314976,0.179906,3,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.140506,-2.034947,0.344013,3,Ethereum
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.172970,-1.200916,-0.040516,3,Litecoin
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.373443,1.322168,-0.487367,0,Dash
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.147776,-2.152861,0.292576,3,Monero
ETC,Ethash,PoW,1.133597e+08,210000000,-0.138948,-2.035041,0.344010,3,Ethereum Classic
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.166175,-1.930259,0.385713,3,ZCash


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [50]:
# Creating a 3D-Scatter with the PCA data and the clusters
import plotly.express as px
fig = px.scatter_3d(
    clustered_df,
    x = "PC1",
    y = "PC2",
    z = "PC3",
    color = "Class",
    symbol = "Class",
    hover_name = "CoinName",
    hover_data = ["Algorithm", "TotalCoinsMined", "TotalCoinSupply"]
)
fig.update_layout(legend=dict(x = 0, y = 1))
fig.show


<bound method BaseFigure.show of Figure({
    'data': [{'customdata': array([['Scrypt', 41.99995383, '42'],
                                   ['Scrypt', 1055184902.04, '532000000'],
                                   ['X13', 29279424622.5027, '314159265359'],
                                   ...,
                                   ['SHA-256', 1999999995.3056, '2000000000'],
                                   ['Scrypt', 14931046.1546605, '250000000'],
                                   ['Scrypt', 128326.99633965, '1000000']], dtype=object),
              'hovertemplate': ('<b>%{hovertext}</b><br><br>Cla' ... '{customdata[2]}<extra></extra>'),
              'hovertext': array(['42 Coin', '404Coin', 'EliteCoin', ..., 'ZEPHYR', 'Gapcoin',
                                  'BitcoinPlus'], dtype=object),
              'legendgroup': '0',
              'marker': {'color': array([0, 0, 0, ..., 0, 0, 0], dtype=int32),
                         'coloraxis': 'coloraxis',
                       

In [53]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns = ['CoinName', 'Algorithm', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply'])

:Table   [CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply]

In [54]:
# Print the total number of tradable cryptocurrencies.
clustered_df['CoinName'].count()

532

In [56]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
from sklearn.preprocessing import MinMaxScaler
cluster_df = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']]
X_minmax = MinMaxScaler().fit_transform(cluster_df)
X_minmax

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [59]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
index_values = (clustered_df.index.tolist())
plot_df = pd.DataFrame(
   data = X_minmax, columns=["TotalCoinSupply_scaled", "TotalCoinsMined_scaled"], index = index_values)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df = plot_df.join(cc_names_df, how = 'inner')

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
class_df = clustered_df['Class']
plot_df = plot_df.join(class_df, how = 'inner')
plot_df.head(10)

,TotalCoinSupply_scaled,TotalCoinsMined_scaled,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,3
ETH,0.000000e+00,0.000109,Ethereum,3
LTC,8.400000e-05,0.000064,Litecoin,3
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,3
ETC,2.100000e-04,0.000115,Ethereum Classic,3
ZEC,2.100000e-05,0.000007,ZCash,3


In [60]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x = "TotalCoinsMined_scaled", y = "TotalCoinSupply_scaled", by = "Class", xlabel = "TotalCryptocurrency Coins Mined", ylabel = "Total Cryptocurrency Coin Supply")


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined_scaled]   (TotalCoinSupply_scaled)